Metin tabanlı bir soru-cevap sistemi oluşturmak için FAISS'i kullandık. FAISS, metin tabanlı soru-cevap sistemimizde, kullanıcı sorularına en alakalı cevapları hızlı ve verimli bir şekilde bulmak için kullandığımız bir araçtır. Metinleri vektörlere dönüştürerek, bu vektörleri indeksleyerek ve benzerlik araması yaparak, sistemin doğru ve hızlı cevaplar vermesini sağlar.

Metin parçaları OpenAIEmbeddings kullanılarak sayısal vektörlere dönüştürüldü.

FAISS, bu vektörleri hızlı bir şekilde aranabilir bir indeks yapısı içinde organize etti. Bu, benzerlik aramalarını hızlandırmak için önemlidir.

Kullanıcı bir soru sorduğunda, soru da bir vektöre dönüştürülür ve FAISS indeksi kullanılarak en benzer metin parçaları bulunur.

In [ ]:
!pip install langchain langchain-openai langchain-community faiss-cpu tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.6
    Uninstalling langchain-text-splitters-0.3.6:
      Successfully uninstalled langchain-text-splitters-0.3.6
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.20
    Uninstalling langchain-0.3.20:
      Successfully uninstalled langchain-0.3.20


In [ ]:
from langchain_openai import OpenAI
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA
import os

In [ ]:
os.environ["OPENAI_API_KEY"] = "your API Key here "

In [ ]:
loader = TextLoader("Rock.txt") #dosyanın içeriğini yükle

In [ ]:
documents = loader.load() #loader nesnesinin load() metodunu çağırarak, belgeleri yükleyip documents değişkenine ata

In [ ]:
documents

[Document(metadata={'source': 'Rock.txt'}, page_content='Rock is a broad genre of popular music that originated in the United States as "rock and roll" in the late 1940s and early 1950s, developing into a range of different styles from the mid-1960s, primarily in the United States and the United Kingdom. It has its roots in rock and roll, a style that drew directly from the black musical genres of blues, rhythm and blues, and country music. \nRock also drew strongly from genres such as electric blues and folk, and incorporated influences from jazz and other musical styles. For instrumentation, rock is typically centered on the electric guitar, usually as part of a rock group with electric bass guitar, drums, and one or more singers. \nUsually, rock is song-based music with a 4 4 time signature and using a verse–chorus form, but the genre has become extremely diverse. Like pop music, lyrics often stress romantic love but also address a wide variety of other themes that are frequently so

In [ ]:
#RecursiveCharacterTextSplitter sınıfını kullanarak bir metin parçalayıcı oluştur
 text_splitter = RecursiveCharacterTextSplitter(
     chunk_size=500, # her parçanın size ı max 500 karakter
     chunk_overlap=0, # parçalar arası örtüşme olmasın
     length_function = len, #uzunluk hesaplama
     )

In [ ]:
docs = text_splitter.split_documents(documents) # documentsdeki verileri text_splitter ile küçük parçalara ayırma

In [ ]:
docs[1] #ayırdığı parçanın 1. indeksini yazdırdık örnek olsun diye

Document(metadata={'source': 'Rock.txt'}, page_content='Rock also drew strongly from genres such as electric blues and folk, and incorporated influences from jazz and other musical styles. For instrumentation, rock is typically centered on the electric guitar, usually as part of a rock group with electric bass guitar, drums, and one or more singers.')

In [ ]:
len(docs)

10

In [ ]:
embeddings = OpenAIEmbeddings() # metinleri sayısal vektöre dönüştürür
#  yani bilgisayarın alayabileceği ve üzerinde işlem yapabileceği sayısal formata dönüştürür

In [ ]:
libray = FAISS.from_documents(docs, embeddings)
# Metinler, FAISS ile hızlı bir şekilde aranabilir bir kütüphane haline getiriliyor.

In [ ]:
Q1 = "When the start alternative Rock"

In [ ]:
Q1_answer = libray.similarity_search(Q1)
#Q1 sorgusuna en benzer metin parçalarını libray indeksinde arar ve bulunan sonuçları Q1_answer adlı bir değişkene atar.

In [ ]:
print(Q1_answer[0].page_content) #Q1 sorgusuna en benzer bulduğu içeriği yazar

From the 1990s, alternative rock began to dominate rock music and break into the mainstream in the form of grunge, Britpop, and indie rock. 
Further fusion subgenres have since emerged, including pop-punk, electronic rock, rap rock, and rap metal. Some movements were conscious attempts to revisit rock's history, including the garage rock/post-punk revival in the 2000s.


In [ ]:
print(Q1_answer[1].page_content) #Q1 sorgusuna biraz benzeyen içeriği yazar

Rock is a broad genre of popular music that originated in the United States as "rock and roll" in the late 1940s and early 1950s, developing into a range of different styles from the mid-1960s, primarily in the United States and the United Kingdom. It has its roots in rock and roll, a style that drew directly from the black musical genres of blues, rhythm and blues, and country music.


In [ ]:
docs_and_scores = libray.similarity_search_with_score(Q1)

In [ ]:
docs_and_scores[0]
#hem benzer metinleri hem de benzerlik skorlarını döndürerek daha detaylı bir sonuç sağlar.

(Document(id='b0beabe4-04ac-4ea9-ab71-5ef664b0b23d', metadata={'source': 'Rock.txt'}, page_content="From the 1990s, alternative rock began to dominate rock music and break into the mainstream in the form of grunge, Britpop, and indie rock. \nFurther fusion subgenres have since emerged, including pop-punk, electronic rock, rap rock, and rap metal. Some movements were conscious attempts to revisit rock's history, including the garage rock/post-punk revival in the 2000s."),
 np.float32(0.33218974))

In [ ]:
docs_and_scores[1]

(Document(id='cb585f6b-45e9-4803-98b2-29e963a6fe7f', metadata={'source': 'Rock.txt'}, page_content='Rock is a broad genre of popular music that originated in the United States as "rock and roll" in the late 1940s and early 1950s, developing into a range of different styles from the mid-1960s, primarily in the United States and the United Kingdom. It has its roots in rock and roll, a style that drew directly from the black musical genres of blues, rhythm and blues, and country music.'),
 np.float32(0.38609308))

In [ ]:
retriever = libray.as_retriever() #FAISS indeksini (libray) bir "Retriever" nesnesine dönüştürür

Retriever, belgeleri bir vektör deposundan almanın bir yoludur. Yani, bu nesne, FAISS indeksine erişmek ve belirli sorgulara göre ilgili belgeleri (metin parçalarını) almak için kullanılır.

In [ ]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)
#İlgili belgeleri bulmak için retriever'ı kullanır, bu belgeleri OpenAI'ın dil modeline iletir ve bir cevap oluşturur.

In [ ]:
retriever_query = "What is the some populer Rock Album"

In [ ]:
results = qa.invoke(retriever_query) #retriever_qury deki soru sorar ve result değişkenine atar

In [ ]:
print(results)

{'query': 'What is the some populer Rock Album', 'result': ' Some popular rock albums include "Sgt. Pepper\'s Lonely Hearts Club Band" by The Beatles, "Dark Side of the Moon" by Pink Floyd, "Nevermind" by Nirvana, and "The Joshua Tree" by U2.'}


In [ ]:
retriever_q1 = "What is the most populer Rock Album"

In [ ]:
results1 = qa.invoke(retriever_q1)

In [ ]:
print(results1)

{'query': 'What is the most populer Rock Album', 'result': " I don't know."}


In [ ]:
libray.save_local("faiss_index_Rock") # daha önce oluşturulan FAISS indeksini (libray) "faiss_index_Rock" adlı bir klasöre kaydeder.

In [ ]:
Rock_saved = FAISS.load_local("faiss_index_Rock", embeddings, allow_dangerous_deserialization=True)
# FAISS indeksini diskten yükler ve Rock_saved değişkeninde saklar, böylece tekrar kullanılır

In [ ]:
qa= RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=Rock_saved.as_retriever())
# önceden kayıt edilen ve yüklenen FAISS indeksini kullanarak yeni bir soru-cevap sistemi oluşturur.

In [ ]:
results = qa.invoke(retriever_query)

In [ ]:
print(results)

{'query': 'What is the some populer Rock Album', 'result': ' Some popular rock albums include "Sgt. Pepper\'s Lonely Hearts Club Band" by the Beatles, "Nevermind" by Nirvana, "Dark Side of the Moon" by Pink Floyd, and "Thriller" by Michael Jackson (which incorporates elements of rock).'}
